<a href="https://colab.research.google.com/github/gabrielcgo/modulo03_BLUE/blob/main/Modulo3_5_projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto 02** / *Módulo Extra*
Essa análise é uma atividade proposta pela Blue EdTech com o objetivo de aplicação prática dos conteúdos ensinados para obtenção parcial da nota (20%) do módulo Extra.

A atividade é composta por 6 questões práticas que serão resolvidas ao decorrer desta apresentação.

Os principais pontos que serão avaliados:

*   Extração de dados
*   Manipulação de dados e criação de gráficos simples com o Pandas
*   Criar um modelo de predição
*   Apresentação dos resultados

In [4]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import f1_score

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
mlb = MultiLabelBinarizer()
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
tfidf_transformer = TfidfTransformer()

# Starting Analysis

Efetivamente vamos iniciar a análise.

**SOBRE ESSE DATASET:**

Os sistemas de submissão de artigos ( CMT , OpenReview , etc.) exigem que os usuários carreguem os títulos dos artigos e os resumos dos artigos e, em seguida, especifiquem as áreas temáticas às quais seus artigos melhor pertencem. Não seria bom se tais sistemas de submissão fornecessem sugestões viáveis ​​de áreas temáticas sobre onde os artigos correspondentes poderiam ser mais bem associados?

Esse conjunto de dados permitiria que os desenvolvedores construíssem modelos de linha de base que poderiam beneficiar esse caso de uso. Os analistas de dados também podem gostar de analisar os meandros
de diferentes artigos e quão bem seus resumos se correlacionam com suas categorias observadas. Além disso, esperamos que o conjunto de dados sirva como uma referência decente para a construção de sistemas úteis de classificação de texto.

***
**RECONHECIMENTOS:**

Thanks to Lukas Schwab (author of arxiv.py) for helping us build our initial data collection utilities. Thanks to Robert Bradshaw for his inputs on the Apache Beam pipeline. Thanks to the ML-GDE program for providing GCP credits that allowed us to run the Beam pipeline at scale on Dataflow.

In [8]:
arxiv_data = pd.read_csv('/content/drive/MyDrive/02/arxiv_data.csv')

In [9]:
arxiv_data.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [10]:
print(f"Existem {len(arxiv_data)} linhas neste dataset.")

Existem 51774 linhas neste dataset.


In [11]:
total_duplicate_titles = sum(arxiv_data["titles"].duplicated())
print(f"Existem {total_duplicate_titles} Títulos duplicados.")

Existem 12802 Títulos duplicados.


In [12]:
contagem = arxiv_data.titles.value_counts()
manter = contagem[contagem > 3]
imdb_dados = arxiv_data[arxiv_data.titles.isin(manter.index)]

titulos = arxiv_data.titles
resumos = arxiv_data.summaries	
termos  = arxiv_data.terms

# Para os títulos:

In [13]:
mlb.fit(titulos)
y = mlb.transform(titulos)

In [14]:
def processamento(tokens):
    
    token_processado = []
    for token in tokens:
        token = token.lower()
        token = lemmatizer.lemmatize(token)
        
        if token not in stop_words:
            token = stemmer.stem(token)
            token_processado.append(token)
        
    return token_processado

documentos = []
for titulo in titulos:
    
    # expressao regular para remover pontuacoes do texto
    titulo = re.sub(r'[^\w\s]','', titulo)
    tokens = processamento(word_tokenize(titulo))
    
    documentos.append(' '.join(tokens))

X_train_counts = vectorizer.fit_transform(documentos)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X = X_train_tfidf

In [15]:
# 1. SVM

split_validation = train_test_split(X, y, titulos, test_size=0.20, random_state=10)

(X_train, X_test)             = split_validation[:2]
(y_train, y_test)             = split_validation[2:4]
(titulos_train, titulos_test) = split_validation[4:]

svm = SGDClassifier() 
clf = OneVsRestClassifier(svm)

clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 27 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 28 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 121 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 123 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 124 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


0.812541935050993


In [ ]:
# 2. Random Forest

split_validation = train_test_split(X, y, titulos, test_size=0.20, random_state=10)

(X_train, X_test)             = split_validation[:2]
(y_train, y_test)             = split_validation[2:4]
(titulos_train, titulos_test) = split_validation[4:]

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))

In [ ]:
# 3. Decision Tree

split_validation = train_test_split(X, y, titulos, test_size=0.20, random_state=10)

(X_train, X_test)             = split_validation[:2]
(y_train, y_test)             = split_validation[2:4]
(titulos_train, titulos_test) = split_validation[4:]

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))

# Para os resumos:

In [16]:
mlb.fit(resumos)
y = mlb.transform(resumos)

def processamento(tokens):
    
    token_processado = []
    for token in tokens:
        token = token.lower()
        token = lemmatizer.lemmatize(token)
        
        if token not in stop_words:
            token = stemmer.stem(token)
            token_processado.append(token)
        
    return token_processado

documentos = []
for resumo in resumos:
    
    # expressao regular para remover pontuacoes do texto
    resumo = re.sub(r'[^\w\s]','', resumo)
    tokens = processamento(word_tokenize(resumo))
    
    documentos.append(' '.join(tokens))

X_train_counts = vectorizer.fit_transform(documentos)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X = X_train_tfidf

In [17]:
# 1. SVM

split_validation = train_test_split(X, y, titulos, test_size=0.20, random_state=10)

(X_train, X_test)             = split_validation[:2]
(y_train, y_test)             = split_validation[2:4]
(titulos_train, titulos_test) = split_validation[4:]

svm = SGDClassifier() 
clf = OneVsRestClassifier(svm)

clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 1 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 66 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 69 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 70 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 73 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sk

0.8710582090372297


In [ ]:
# 2. Random Forest

split_validation = train_test_split(X, y, titulos, test_size=0.20, random_state=10)

(X_train, X_test)             = split_validation[:2]
(y_train, y_test)             = split_validation[2:4]
(titulos_train, titulos_test) = split_validation[4:]

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))

In [ ]:
# 3. Decision Tree

split_validation = train_test_split(X, y, titulos, test_size=0.20, random_state=10)

(X_train, X_test)             = split_validation[:2]
(y_train, y_test)             = split_validation[2:4]
(titulos_train, titulos_test) = split_validation[4:]

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))

Compare os resultados obtidos dos titles e abstracts. Qual atributo é mais discriminativo?

R = Abstracts (Resumos), pois obteve resultados maiores de F1. Provavelmente isso ocorreu pois os resumos são maiores que os títulos, contribuindo para o aumemto das entradas para treino e teste. Porém o custo benefício entre o tempo de processamento nas duas variáveis talvez não justifique o ganho de menos de 10% do F1 score, é um ganho baixo de um sobre o outro que pode não fazer sentido dependendo do dataset.